In [1]:
from stochastic_volatility_models.src.models.heston.heston import Heston
from notebook_utils import Notebook
import numpy as np

2024-09-03 11:01:40.091 | INFO     | stochastic_volatility_models.config:initialise:30 - Loaded module `stochastic_volatility_models` from project directory path `/Users/mayurankv/Documents/Mayuran/Programming/Projects/Academic/Imperial College London/MSc Statistics/Dissertation/Project/modules`


In [4]:
# time = "2013-07-11"
# time = "2015-09-01"
time = "2017-11-30"
vix = False
nb = Notebook(
	time=time,
	model=Heston(parameters={}),
	use_optimal_spx_params=not vix,
)

print(nb.model.convert_parameters_to_market_measure(nb.time, nb.underlying, nb.volatility_underlying))

simulation_length = 1 / 2
historical_period = 0.1
forecast_confidences = [0.9, 0.95, 0.99]
performance, prediction_interval_accuracies = nb.plot_forecast(
	simulation_length=simulation_length,
	historical_period=historical_period,
	forecast_confidences=forecast_confidences,
	config={
		"toImageButtonOptions": {
			"format": "svg",  # one of png, svg, jpeg, webp
			"filename": "heston intervals",
			"height": 400,
			"width": 400,
			"scale": 1,  # Multiply title/legend/axis/canvas sizes by this factor
		}
	},
)

print(performance)
print(prediction_interval_accuracies)
print(" & ".join([str(round(a * 1000, 3)) for a in performance.values()]))

(-5.75061619522283, 0.0)


{'MAE': 0.0028750378589953653, 'RMSE': 0.00452474805872525}
{0.9: 0.9682539682539683, 0.95: 0.9761904761904762, 0.99: 1.0}
2.875 & 4.525


In [33]:
print("Theoretical: ", np.sqrt(nb.model.parameters["long_term_variance"] / 252))
print("Empirical  : ", (nb.volatility_forecast[("Forecast", "Mean")].iloc[-1]))
# print("vf         : ", (vf[("Forecast", "Mean")].iloc[-1]))

Theoretical:  0.005776074055938317
Empirical  :  0.005014988422289179
vf         :  0.0026321704421344294


In [5]:
nb = Notebook(
	time=time,
	model=Heston(parameters={}),
	use_optimal_spx_params=False,
)
print(nb.model.parameters)
print("w_vol=0: ", nb.evaluate_fit(vol_weight=0.0, skew_weight=0.0), "\nw_vol=1: ", nb.evaluate_fit(vol_weight=1.0, skew_weight=0.0))

{'initial_variance': 0.007215276601791483, 'long_term_variance': 0.0363301387547702, 'volatility_of_volatility': 0.6813592590835942, 'mean_reversion_rate': 1.6177774153739044, 'wiener_correlation': -0.2851318745267004}
w_vol=0:  0.03889943667332727 
w_vol=1:  0.18284429645656397


In [11]:
from stochastic_volatility_models.visualisations.volatility_surface import plot_volatility_surface_comparison, plot_volatility_surface_error_comparison

nbe = Notebook(
	time=time,
	model=None,
)

config = {
	"toImageButtonOptions": {
		"format": "svg",  # one of png, svg, jpeg, webp
		"filename": "custom_image",
		# 'height': 800,
		# 'width': 1000,
		"scale": 1,  # Multiply title/legend/axis/canvas sizes by this factor
	}
}

model = nb.plot_surfaces(volatility=False)
empirical = nbe.plot_surfaces(volatility=False)
plot_volatility_surface_comparison(model, empirical, 0.4).show(config=config)
plot_volatility_surface_error_comparison(model, empirical, 1).show(config=config)
model = nb.plot_surfaces(volatility=True, call=True, price=False)
empirical = nbe.plot_surfaces(volatility=True, call=True, price=False)
plot_volatility_surface_comparison(model, empirical, 0.4).show(config=config)
plot_volatility_surface_error_comparison(model, empirical, 1).show(config=config)

/Users/mayurankv/Documents/Mayuran/Programming/Projects/Academic/Imperial College London/MSc Statistics/Dissertation/Project/modules/stochastic_volatility_models/stochastic_volatility_models/visualisations/volatility_surface.py:152: RuntimeWarning:

divide by zero encountered in divide



In [8]:
from stochastic_volatility_models.src.models.time_series.heterogeneous_autoregressive_model import HAR
from stochastic_volatility_models.visualisations.forecasts import plot_forecast_comparison, plot_forecast

har = HAR()

har.fit(
	time=nb.time,
	underlying=nb.underlying,
	fitting_period=3,
)

forecast_period = simulation_length
forecast_confidences = [0.9, 0.95, 0.99]
har.forecast(forecast_period=forecast_period, forecast_confidences=forecast_confidences)

config = {
	"toImageButtonOptions": {
		"format": "svg",  # one of png, svg, jpeg, webp
		"filename": "comparison_plot",
		# "height": 300,
		# "width": 1000,
		"scale": 1,  # Multiply title/legend/axis/canvas sizes by this factor
	}
}

plot_forecast_comparison(nb.underlying, nb.volatility_forecast, har.volatility_forecast, 1, "Heston Forecast", "log HAR-RV Forecast").show(config=config)

In [6]:
config = {
	"toImageButtonOptions": {
		"format": "svg",  # one of png, svg, jpeg, webp
		"filename": "har_intervals",
		"height": 400,
		"width": 400,
		"scale": 1,  # Multiply title/legend/axis/canvas sizes by this factor
	}
}

plot_forecast(
	underlying=nb.underlying,
	volatility_forecast=nb.volatility_forecast,
	historical_period=0.1,
).show(config=config)

In [23]:
from scipy.stats import ncx2
from scipy.integrate import quad_vec
from stochastic_volatility_models.visualisations.forecasts import plot_forecast

vf = nb.volatility_forecast.copy()

T = np.arange(1 / 252, 1 / 2, 1 / 252)[:, np.newaxis]

c = 2 * nb.model.parameters["mean_reversion_rate"] / ((1 - np.exp(-nb.model.parameters["mean_reversion_rate"] * T)) * (nb.model.parameters["volatility_of_volatility"] ** 2))
df = (4 * nb.model.parameters["mean_reversion_rate"] * nb.model.parameters["long_term_variance"]) / (nb.model.parameters["volatility_of_volatility"] ** 2)
nc = 2 * c * np.exp(-nb.model.parameters["mean_reversion_rate"] * T) * nb.model.parameters["initial_variance"]


def integrand(x):
	return np.sqrt(x) * ncx2.pdf(x, df, nc)


mean_sqrt, error = quad_vec(integrand, 0, np.inf)
mean = mean_sqrt / np.sqrt(252 * 2 * c)


q = np.array([0.95, 0.975, 0.995])

vf["Upper"] = np.concatenate([[[vf[("Forecast", "Mean")].iloc[0]] * 3], np.sqrt(ncx2.ppf(q=[q], df=df, nc=nc) / (2 * c * 252))])
vf["Lower"] = np.concatenate([[[vf[("Forecast", "Mean")].iloc[0]] * 3], np.sqrt(ncx2.ppf(q=[1 - q], df=df, nc=nc) / (2 * c * 252))])
vf[("Forecast", "Mean")] = np.concatenate([[[vf[("Forecast", "Mean")].iloc[0]]], mean])
plot_forecast(
	underlying=nb.underlying,
	volatility_forecast=vf,
	historical_period=0.1,
).show()

plot_forecast(
	underlying=nb.underlying,
	volatility_forecast=nb.volatility_forecast,
	historical_period=0.1,
).show()

$$
P[\frac{\sigma_{t}}{\sqrt{252}}\leq\gamma]=P[\sigma_{t}\leq\sqrt{252}\gamma]=P[2c\sigma^{2}_{t}\leq252\gamma^{2}2c]=P[Y_t\leq252\gamma^{2}2c]
$$

$$
E[\frac{\sigma}{\sqrt{252}}]=\frac{1}{\sqrt{252}}E[\sqrt{\sigma^{2}_{t}}]=\frac{1}{\sqrt{252}}E[\sqrt{\frac{Y_{t}}{2c}}]=\frac{1}{\sqrt{252\cdot 2c}}E[\sqrt{Y_{t}}]
$$

In [4]:
# nb.model.parameters = {"initial_variance": 0.0895791230480412, "long_term_variance": 0.021357008010883253, "volatility_of_volatility": 2.969506343913128, "mean_reversion_rate": 9.443471596806445, "wiener_correlation": -0.9128602034942876}
# nb.model.parameters = {"initial_variance": 0.0895791230480412, "long_term_variance": 0.0585764748315325, "volatility_of_volatility": 0.66658276362568, "mean_reversion_rate": 2.8425708029119816, "wiener_correlation": -0.987091552060316}
# nb.model.parameters = {	"initial_variance": 0.01417449066157833,	"long_term_variance": 0.0976082312303378,	"volatility_of_volatility": 0.8599397768293384,	"mean_reversion_rate": 0.24175166704773687,	"wiener_correlation": -0.4525348016611488}  # {'initial_variance': 0.01417449066157833, 'long_term_variance': 0.0976088190987323, 'volatility_of_volatility': 0.8599436770559035, 'mean_reversion_rate': 0.2417514622062309, 'wiener_correlation': -0.45253515792269194}
# nb.model.parameters = {'initial_variance': 0.01417449066157833, 'long_term_variance': 0.052396679044242694, 'volatility_of_volatility': 0.85478037502723, 'mean_reversion_rate': 0.198158055919218, 'wiener_correlation': -0.4723537255278572}
# nb.model.parameters = {'initial_variance': 0.01417449066157833, 'long_term_variance': 0.052396679044242694, 'volatility_of_volatility': 0.45478037502723, 'mean_reversion_rate': 1.198158055919218, 'wiener_correlation': -0.9123537255278572}
# nb.model.parameters = {'initial_variance': 0.01417449066157833, 'long_term_variance': 0.05239668798427822, 'volatility_of_volatility': 0.45478042033659316, 'mean_reversion_rate': 5.198158114962719, 'wiener_correlation': -0.9123537362602547}
# nb.model.parameters = {'initial_variance': 0.00912105589815607, 'long_term_variance': 0.04269903151646093, 'volatility_of_volatility': 0.9889425869966404, 'mean_reversion_rate': 2.6587470619088265, 'wiener_correlation': -0.8726926166561948}
# nb.model.parameters = {"initial_variance": 0.0895791230480412, "long_term_variance": 0.0285764748315325, "volatility_of_volatility": 2.66658276362568, "mean_reversion_rate": 5.8425708029119816, "wiener_correlation": -0.987091552060316}
# nb.model.parameters = {"initial_variance": 0.01417449066157833, "long_term_variance": 0.052396679044242694, "volatility_of_volatility": 0.85478037502723, "mean_reversion_rate": 0.198158055919218, "wiener_correlation": -0.4723537255278572}
